#  Models with Linearizable Nonlinear Objectives

Nonlinearity is often unavoidable, but some frequently occurring nonlinear objective functions are exceptions. We introduce in this section those piecewise linear, minimax, maximin, and absolute value objective functions which though nonlinear at first glance can be modeled with a linear objective function and linear constraints.

## Remind: Linear Programming and Polyhedra

Linear programs have polyhedral feasible sets:

$$\{x|Ax \leq b\}$$

As a polyhera:

![](poly1.png)

So, Can every polyhedron be expressed $Ax\leq b$?

Not this one:

![](poly2.png)

=> This is non-convex polyhera

### Convex polyhedra

Remark:

If $x,y\in R^n$, then the combination:
$$w=\alpha x+(1-\alpha)y$$
for some $0<\alpha <1$ is called **convex combinations**. It’s the line segment that connects x and y

> Theorem: If $Ax\leq b$ and $Ay\leq b$, and w is a convex combination of x and y, then $Aw\leq b$.

**Proof:** Suppose $w=\alpha x+(1-\alpha)y$
\begin{align}
Aw&=A(\alpha x+(1-\alpha)y)\\
&=\alpha Ax + (1-\alpha)Ay\\
&\leq \alpha b+(1-\alpha)b\\
&=b
\end{align}

Thus, $Aw\leq b$.

The previous result implies that every polyhedron describe able as $Ax\leq b$ must contain all convex combinations of its points.
* Such polyhedra are called **convex**
* Informal definition:  if you were to “shrink-wrap” it, the entire polyhedron would be covered with no extra space.

>Goes  the  other  way  too: every  convex  polyhedron  can  be represented as $Ax\leq b$ for appropriately chosen A and b.

## Piecewise linear functions

Some problems do not appear to be LPs but can beconverted to LPs using a suitable transformation. An important case will be **convex piecewise linear functions**.

Consider the following **non-linear** optimization:
\begin{align}
&\min_x \quad f(x)\\
&\text{s.t.} \\
&\qquad x\geq 0
\end{align}

where $f(x)$ is:

![](pw1.png)

The trick is to convert the problem into **epigraph** form: add an extra decision variable t and turn the cost into a constraint.

\begin{align}
&\min_{x,t} \quad t\\
&\text{s.t.} \\
&\qquad t\geq f(x)\\
&\qquad x\geq 0
\end{align}

![](pw2.png)

This feasible set is **polyhedral**.  It is the set of $(x,t)$ satisfying:
$$\{t\geq -2x+3, t\geq -\frac{1}{2}x+\frac{3}{2}, t\geq 3x-9\}$$

The linear programming:

\begin{align}
&\min_{x,t} \quad t\\
&\text{s.t.} \\
&\qquad t\geq -2x+3\\
&\qquad t\geq -\frac{1}{2}x+\frac{3}{2}\\
&\qquad t\geq 3x-9\\
&\qquad x\geq 0
\end{align}

**Julia Code**

In [1]:
using JuMP, GLPK, LinearAlgebra

m = Model(with_optimizer(GLPK.Optimizer))
@variable(m, t)
@variable(m, x>=0)
@constraint(m, t >= -2*x+3)
@constraint(m, t >= -1/2*x+3/2)
@constraint(m, t >= 3*x-9)
@objective(m, Min, t)
m

A JuMP Model
Minimization problem with:
Variables: 2
Objective function type: VariableRef
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: t, x

In [3]:
optimize!(m)
println("x = ", value(x))
println("objective = ", objective_value(m))

x = 3.0
objective = 0.0


>Epigraph trick only works if it’s a convex polyhedron.

![](pw3.png)

This epigraph is not a **convex polyhedron** so it cannot be the feasible set of a linear program.

## Minimax and Maximin Problem

The maximum of several linear functions is always **convex**.So we can minimize it using the epigraph trick.  Example:

$$f(x)=\max_{i=1,..,k}\{a_i^Tx+b_i\}$$

![](mini1.png)

\begin{align}
&\min_x \quad \max_{i=1,..,k}\{a_i^Tx+b_i\}
\end{align}

=> 
\begin{align}
&\min_{x,t} \quad t\\
&\text{s.t.} \\
&\qquad t\geq a_i^Tx+b_i \quad \forall i\\
\end{align}

Similarly, the maximin problem:

![](mini2.png)

\begin{align}
&\max_x \quad \min_{i=1,..,k}\{a_i^Tx+b_i\}
\end{align}

=> 
\begin{align}
&\max_{x,t} \quad t\\
&\text{s.t.} \\
&\qquad t\leq a_i^Tx+b_i \quad \forall i\\
\end{align}

**NOTE**: As you can see in two above figures, minimax != maximin

### Chebyshev center

What is the largest sphere you can fit inside a polyhedron?

If $y$ is the center, then draw perpendicular lines to each face of the polyhedron. We want to maximize the smallest $d_i$.  In other words,

\begin{align}
&\max_y \quad \min_{i=1,..,k}d_i(y)
\end{align}

![](cheb.png)

The optimal $y$ is the **Chebyshev center**

To compute the distance between $y$ and the hyperplane $a^Tx=b$, notice that if the distance is $r$, then $y+\frac{r}{||a||}a$ belong to the hyperplane:
$$a^T\big(y+\frac{r}{||a||}a\big)=b$$
Simplifying, we obtain:
$$a^Ty+||a||r=b$$
There, the statement “The distance between $y$ and each hyperplane is at least $r$” is equivalent to 
$$a_i^Ty+||a_i||r\leq b_i \quad \forall i$$

The transformation of **Chebyshev center problem** to an LP is given by:
\begin{align}
&\max_y \quad \min_{i=1,..,k}d_i(y)\\
&\text{s.t.} \\
&\qquad a_i^Ty \leq b_i \quad \forall i\\
\end{align}
=> 
\begin{align}
&\max_{y,r} r\\
&\text{s.t.} \\
&\qquad a_i^Ty+||a_i||r\leq b_i \quad \forall i\\
\end{align}

**Example:** Find the Chebyshev center of the polyhedrondefined by the following inequalities:
$$2x−y+ 2z\leq 2,−x+ 2y+ 4z\leq 16,x+ 2y−2z\leq 8$$
$$x,y,z \geq 0$$

![](cheb2.png)

**Julia Code**

In [5]:
A = [2 -1 2; -1 2 4; 1 2 -2; -1 0 0; 0 -1 0; 0 0 -1];
b = [2; 16; 8; 0; 0; 0]
m = Model(with_optimizer(GLPK.Optimizer))
@variable(m, r >= 0)           # radius
@variable(m, x[1:3])           # coordinates of center
for i = 1:size(A,1)
    @constraint(m, A[i,:]'*x + r*norm(A[i,:]) <= b[i])
end
@objective(m, Max, r)     # maximize radius
m

A JuMP Model
Maximization problem with:
Variables: 4
Objective function type: VariableRef
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 6 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: r, x

In [11]:
optimize!(m)
println("The coordinates of the Chebyshev center are: ", value(x[1]),',',value(x[2]),',',value(x[3]))
println("The largest possible radius is: ", objective_value(m))

The coordinates of the Chebyshev center are: 0.75,3.2500000000000004,0.75
The largest possible radius is: 0.75


## Absolute values

Absolute values are piecewise linear, we can treat them by:

\begin{align}
&\min_x |x|\\
&\text{s.t.} \\
&\qquad Ax \leq b\\
\end{align}
=> 
\begin{align}
&\min_{x,t} t\\
&\text{s.t.} \\
&\qquad Ax \leq b\\
&\qquad t \geq x \\
&\qquad t \geq -x
\end{align}

So are sums of absolute values:
\begin{align}
&\min_{x,y} |x|+|y|\\
\end{align}
=> 
\begin{align}
&\min_{x,y,t,r} t+r\\
&\text{s.t.} \\
&\qquad t \geq x \\
&\qquad t \geq -x\\
&\qquad r \geq y \\
&\qquad r \geq -y
\end{align}

In [3]:
using GLPK, JuMP, LinearAlgebra
m = Model(with_optimizer(GLPK.Optimizer))
@variable(m, t)
@variable(m, x)
@constraint(m, 2*x>=4)
@constraint(m,t>=x)
@constraint(m,t>=-x)
optimize!(m)
println(value(t))
println(value(x))

2.0
2.0
